# California PPE Projections

## California Burn Rates

Burn rates extrapolated from CHA memo. For example, CA assumes N95 are single use per covid patient, and 3 staff visits per hour per COVID patient, so (24)(3) = 72 N95 masks per patient per day.

Small, reasonable changes in assumption can drastically change burn rates by up to 20X. For example, JHU assumes 6 N95 changes per patient per day for ICU patients, and 2.6 for non-ICU. So with the IHME assumption that 27% of patients go to the ICU, (0.27)(6) + (0.73)(2.6) = 3.52 N95 masks per patient per day.

In [1]:
import sys
import pandas as pd
import numpy as np
from restart import RestartModel
from restart import Data
from restart.util import set_config, to_df, to_sheet, display_population, format_population, format_cells
import ipywidgets as widgets
import bqplot
from bqplot import pyplot as plt

config = set_config('../config/ca-vent')
restart = RestartModel(config_dir='../config/ca-vent', population='dict')
model = restart.model
chart_colors = ["#77AADD", "#99DDFF", "#44BB99", "#BBCC33", "#AAAA00", "#EEDD88",
                "#EE8866", "#FFAABB", "DDDDDD"]

def generate_group_bar(df, title="", scientific_notation=False):
    fig=plt.figure(title=title)
    
    bar_chart = plt.bar(x=df.columns.values.tolist(),
                        y=df,
                        labels=df.index.values.tolist(),
                        display_legend=False)
    bar_chart.type = "grouped"
    bar_chart.colors = chart_colors[:df.index.values.size]
    if df.columns.name:
        plt.xlabel(df.columns.name.rsplit(" ", 1)[0])
    plt.ylim(0, np.amax(df.values))
    if scientific_notation == False:
        fig.axes[1].tick_format = "0.0f"
    return fig

def generate_stacked_bar(df, title="", scientific_notation=False):
    fig=plt.figure(title=title)
    
    bar_chart = plt.bar(x=df.columns.values.tolist(),
                        y=df,
                        labels=df.index.values.tolist(),
                        display_legend=False)
    bar_chart.type = "stacked"
    bar_chart.colors = chart_colors[:df.index.values.size]
    if df.columns.name:
        plt.xlabel(df.columns.name.rsplit(" ", 1)[0])
    plt.ylim(0, np.amax(df.values))
    if scientific_notation == False:
        fig.axes[1].tick_format = "0.0f"
    return fig

def generate_html_legend(df, colors=chart_colors, table=True, font_size=13):
    name = df.index.name.rsplit(" ", 1)[0]
    html_string = f"<div style='font-size:{font_size}px; font-family:helvetica'><b style='font-weight:bold'>{name}</b>"
    indices = df.index.values
    if table:
        html_string += "<table><tr>"
        for i in range(0, indices.size):
            if i % 2 == 0:
                index_num = int(i / 2)
            else:
                index_num = int(i / 2 + indices.size / 2)
            html_string += f"<td style='padding:0 5px'><span style='color:{colors[index_num]}'>█</span> {indices[index_num]}</td>"
            if i % 2 != 0:
                html_string += "</tr><tr>"
        html_string += "</tr></table>"
    else:
        for_count = 0
        for string in indices:
            html_string += f"<br><span style='color:{colors[for_count]}'>█</span> {string}"
            for_count += 1
    html_string += "</div>"
    return widgets.HTML(html_string)

def generate_group_bar_legend(df, title="", scientific_notation=False):
    chart = generate_group_bar(df, title=title, scientific_notation=scientific_notation)
    legend = generate_html_legend(df, table=True)
    return widgets.VBox([chart, legend])

def generate_stacked_bar_legend(df, title="", scientific_notation=False):
    chart = generate_stacked_bar(df, title=title, scientific_notation=scientific_notation)
    legend = generate_html_legend(df, table=True)
    return widgets.VBox([chart, legend])

burn_sheet = format_population(to_sheet(model.demand.demand_per_unit_map_dn_um.df))
burn_chart = generate_stacked_bar_legend(model.demand.demand_per_unit_map_dn_um.df)
display(burn_sheet)
display(burn_chart)

Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format='0,000', read_only=True, row_end=1, row_start=0…

## California Stockpile Projections
Based on an ensemble of differing epidemiological models.

In [2]:
import ipywidgets as widgets
from scipy.stats import norm
import math

def triangular(a,b,c):
    return math.sqrt( ((a*a + b*b + c*c) - a*b - a*c - b*c) / 18 )

ca_epi_df = pd.read_csv('epi_ranges.csv',index_col='Model')
# ca_epi_df.drop(['CA Projected Non-COVID Patients'], axis=0, inplace=True)

# adjusting so it's in terms of population, since we're not doing vents here
ca_epi_df = ca_epi_df * 4.12
ca_epi_df.columns = ['Population Low', 'Population Mid', 'Population High']
ca_stdev = ca_epi_df.apply(
    lambda row: triangular(
        row['Population Mid'],
        row['Population Low'], 
        row['Population High']
    ), axis=1
)

ca_epi_df['Population Mean'] = ca_epi_df.mean(axis=1)

ca_epi_df['Population SD'] = ca_stdev


def calc_eoq(df, cr):
    Z=norm.ppf(cr)
    df['Adjusted Population'] = df['Population Mean'] + (Z * df['Population SD'])
    return df

def display_ca_eoq(cr, days):
    # calculate the hospitalization EOQ
    epi = calc_eoq(ca_epi_df, cr)
    eoq_df = 0.243 * epi
    # adjusting the non-COVID patients
    eoq_sheet = to_sheet(eoq_df)
    display_population(eoq_sheet, round=True)
    # calculate stockpile projections
    preds_df = Data(
        "demand_by_pop_total_pn_tc", config)
    preds_df.array = (model.demand.demand_by_pop_per_person_pn_uc.array.T * ca_epi_df["Adjusted Population"].to_numpy().T).T
    preds_df.array *= days
    preds_df.df.drop(['Ventilators'], axis=1, inplace=True)
    preds_sheet = to_sheet(preds_df.df)
    chart = generate_group_bar_legend(preds_df.df, scientific_notation=True)
    display_population(preds_sheet, round=True)
    display(chart)
    
ca_burn_sheet = format_population(to_sheet(model.demand.demand_per_unit_map_dn_um.df))

ca_cr_slider = widgets.FloatSlider(min=0.70,max=0.99,step=0.01,value=0.95, continuous_update=False, description="CR")
ca_day_slider = widgets.IntSlider(min=1, max=120, value=30, continuous_update=False, description="Days")
ca_out = widgets.interactive_output(display_ca_eoq, {'cr': ca_cr_slider, 'days': ca_day_slider})
widgets.VBox([ca_cr_slider, ca_day_slider, ca_out])

## OES Projections

Using WA tiering system, BLS OES population statistics, and WA burn rate assumptions.

In [3]:
oes_config = set_config('../restart')
oes_restart = RestartModel(
    config_dir='../restart',
    data_dir='../../data/ingestion',
    population='oes',
    state='California',
    subpop='healthcare')
oes_model = oes_restart.model
oes_model.inventory.set_average_orders_per_period(oes_model.demand.demand_by_popsum1_total_rp1n_tc)

oes_slider = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)

def dashboard(backstop):
    set_stock(backstop)
    
def display_stock(df):
    df_round = df.round()
    index_name = "Population"
    headers = ['Essential', 'Non-Essential']
    df_round.insert(loc=0, column=index_name, value=headers)
    sheet = to_sheet(df_round)
    format_cells(sheet)
    sheet.row_headers = False
    df_chart = df
    df_chart.index = df_chart.index.get_level_values(1)
    chart = generate_group_bar_legend(df_chart)
    display(sheet)
    display(chart)
    
def set_stock(backstop):
    backstop = [backstop]
    oes_model.inventory.order(oes_model.inventory.inv_by_popsum1_total_rp1n_tc)
    oes_model.inventory.set_min_in_periods(backstop)
    display_stock(oes_model.inventory.inv_by_popsum1_total_rp1n_tc.df)
    
wa_burn_sheet = format_population(to_sheet(oes_model.demand.demand_per_unit_map_dn_um.df))
oes_pop = format_population(to_sheet(oes_model.population.population_pP_tr.df))
    
oes_out = widgets.interactive_output(dashboard, {'backstop': oes_slider})

widgets.VBox([oes_slider, oes_out])

## JHU Burn Rates

### ICU Assumptions

Gloves: 2 gloves for each of 170 changes per patient per day. This assumes a change with each patient encounter, as per normal practice by all healthcare workers.

Gowns: Assumes that COVID patients are cohorted and that a single gown is worn for 4 hours by each healthcare worker assigned to the COVID cohort, unless it becomes visibly soiled. Accounting for all the various healthcare workers involved in the care of an ICU patient, 20 changes per patient per day.

Simple masks: Assumes that COVID patients are cohorted and that a single mask is worn for 4 hours by each healthcare worker assigned to the COVID cohort, unless it becomes visibly soiled. 10 changes per patient per day.

N95 respirators: Assumes that COVID patients are cohorted and that a single mask is worn for 4 hours by each healthcare worker assigned to the COVID cohort, unless it becomes visibly soiled. N95s are used only by healthcare workers in proximity (3 feet) to COVID patients. 6 changes per patient per day.

### Non-ICU Assumptions

Gloves: 2 gloves for each of 80 changes per patient per day. This assumes a change with each patient encounter, as per normal practice by all healthcare workers.

Gowns: Assumes that COVID patients are cohorted and that a single gown is worn for 4 hours by each healthcare worker assigned to the COVID cohort, unless it becomes visibly soiled. 20 changes per patient per day.

Simple masks: Assumes that COVID patients are cohorted and that a single mask is worn for 4 hours by each healthcare worker assigned to the COVID cohort, unless it becomes visibly soiled. 10 changes per patient per day.

N95 respirators: Respirators worn only for nebulizer treatments, intubations, and other aerosol-generating procedures. An average of 2.6 changes per patient per day.

### Daily Burn Rates

Let `n` be the number of hospital COVID patients. IHME assumption is that 27% of patients will go to ICU.

Gloves (g): 

`g = (0.27n)(85) + (0.73n)(40) = 52.15 pairs per patient per day`

Gowns: (G): 

`G = 20n = 20 gowns per patient per day`

Simple masks (m): 

`m = 10n = 10 masks per patient per day`

N95 Respirators (M): 

`M = (0.27n)(6) + (0.73n)(2.6) = 3.52 N95 masks per patient per day`





In [4]:
jhu_config = set_config('../config/jhu')
jhu_restart = RestartModel(config_dir='../config/jhu', population='dict')
jhu_model = jhu_restart.model

jhu_epi_df = pd.read_csv('epi_ranges.csv',index_col='Model')
jhu_epi_df.drop(['CA Projected Non-COVID Patients'], axis=0, inplace=True)

# adjusting so it's in terms of population, since we're not doing vents here
jhu_epi_df = jhu_epi_df * 4.12
jhu_epi_df.columns = ['Population Low', 'Population Mid', 'Population High']
jhu_stdev = jhu_epi_df.apply(
    lambda row: triangular(
        row['Population Mid'],
        row['Population Low'], 
        row['Population High']
    ), axis=1
)

jhu_epi_df['Population Mean'] = jhu_epi_df.mean(axis=1)

jhu_epi_df['Population SD'] = jhu_stdev


def calc_eoq(df, cr):
    Z=norm.ppf(cr)
    df['Adjusted Population'] = df['Population Mean'] + (Z * df['Population SD'])
    return df

def display_jhu_eoq(cr, days):
    # calculate the hospitalization EOQ
    epi = calc_eoq(jhu_epi_df, cr)
    eoq_df = 0.243 * epi
    # adjusting the non-COVID patients
    eoq_sheet = to_sheet(eoq_df)
    display_population(eoq_sheet, round=True)
    # calculate stockpile projections
    preds_df = Data(
        "demand_by_pop_total_pn_tc", jhu_config)
    preds_df.array = (jhu_model.demand.demand_by_pop_per_person_pn_uc.array.T * jhu_epi_df["Adjusted Population"].to_numpy().T).T
    preds_df.array *= days
    preds_sheet = to_sheet(preds_df.df)
    chart = generate_group_bar_legend(preds_df.df, scientific_notation=True)
    display_population(preds_sheet, round=True)
    display(chart)
    
jhu_burn_sheet = format_population(to_sheet(jhu_model.demand.demand_per_unit_map_dn_um.df))

jhu_cr_slider = widgets.FloatSlider(min=0.70,max=0.99,step=0.01,value=0.95, continuous_update=False, description="CR")
jhu_day_slider = widgets.IntSlider(min=1, max=120, value=30, continuous_update=False, description="Days")
jhu_out = widgets.interactive_output(display_jhu_eoq, {'cr': jhu_cr_slider, 'days': jhu_day_slider})
widgets.VBox([jhu_cr_slider, jhu_day_slider, jhu_out])

# For Joey to Graph
Visual summary of all burn-rate assumptions

In [13]:
burn_summary_df = pd.concat([
    jhu_model.demand.demand_per_unit_map_dn_um.df, 
    oes_model.demand.demand_per_unit_map_dn_um.df, 
    model.demand.demand_per_unit_map_dn_um.df
])

burn_summary_df.fillna(0, inplace=True)

burn_summary_sheet = format_population(to_sheet(burn_summary_df))

burn_summary_sheet

Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format='0,000', read_only=True, row_end=9, row_start=0…

In [14]:
import ipyvuetify as v
v.theme.themes.light.primary = 'colors.teal'

v.Tabs(_metadata={'mount_id': 'content-main'}, children=[
    v.Tab(children=['PPE Analysis']),
    v.Tab(children=['Population']),
    v.Tab(children=['Burn Rates']),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, xl4=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Covid Incremental Ventilator Requirements"]),
                v.CardText(children=[
                    "Vent + PPE analysis with CA burn rates:"
                ]),
                ca_cr_slider, ca_day_slider, ca_out,
                v.CardText(children=[
                    "PPE and population analysis with JHU burn rates:"
                ]),
                jhu_cr_slider, jhu_day_slider, jhu_out,
                v.CardText(children=[
                    "PPE and population analysis with OES population and WA burn rates:"
                ]),
                oes_slider, oes_out,
            ]),
                
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, xl4=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["BLS Population with High Granularity"]),
                v.CardText(children=[
                    "OES Population"
                ]),
                oes_pop
            ])
                
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, x14=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Burn Rates"]),
                v.CardSubtitle(children=["Per Capita Resource Demand: CA Assumptions"]),
                burn_sheet,
                burn_chart,
                v.CardSubtitle(children=["Per Capita Resource Demand: JHU Assumptions"]),
                jhu_burn_sheet,
                v.CardSubtitle(children=["Per Capita Resource Demand: WA Assumptions"]),
                wa_burn_sheet,
            ])
        ])
    ])
])

Tabs(children=[Tab(children=['PPE Analysis']), Tab(children=['Population']), Tab(children=['Burn Rates']), Tab…